# Message Authentication Codes

# Introduction

Message Authentication Codes (MACs) are used to address two main information security objectives:

- **Authentication**. 
    - Is the message received by Bob really from Alice?
- **Data Intgerity** 
    - Has the received message been altered during transmission?

To verify these, Alice sends an appendix $MAC$ to along with the message $m$ as $(m, MAC)$.

There are different kinds of MACs but a specific kind of MAC we will be focusing on is the Hashed MAC (HMAC) which is defined as 

$HMAC(m,K) = H((K \oplus opad) \Vert H((K \oplus ipad)\Vert m))$ where 
- opad and ipad are paddings, 
- $K$ is the key, and 
- $H$ is a hash function.




## References

- 